# Benchmarking Our Algorithm against others

In [75]:
import sys
sys.path.append('../')
from panav.environment.env import Room
from panav.hybrid import HybridGraph
from panav.util import flowtime,makespan
from panav.PBS.PBS import PBS
from panav.TrafficAwarePlanning import TAHP
from panav.sequential import sequential_planning
from panav.PIBT import PIBT_plan
from panav.SAMP.solvers import Tube_Planning



from time import time
import pickle as pkl

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:

bloating_r = 0.5
vmax = 1.0

#### 100 agent configuration ####
n_agents = list(range(10,101,10))
bloating_r = 0.5
vmax = 1.0



TIMEOUT = 100 # timeout

# Algorithms
algs = {  
        "PIBT": lambda hg: PIBT_plan(hg,vmax,bloating_r,TIMEOUT,simple_plan=False),
        "S2M2": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Simple_MILP_Planning",TIMEOUT=TIMEOUT)[0],
        "PBS_Tube": lambda hg: PBS(hg.env,vmax=vmax,bloating_r=bloating_r,low_level_planner="Tube_Planning",TIMEOUT=TIMEOUT)[0],
        "TAHP": lambda hg: TAHP(hg,vmax=vmax,bloating_r=bloating_r,TIMEOUT=TIMEOUT),
        "Seq_Tube": lambda hg: sequential_planning(Tube_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT),      
        # "Seq_Temp_Obs": lambda hg: sequential_planning(Simple_MILP_Planning,hg.env,vmax,bloating_r,TIMEOUT=TIMEOUT)
        }
env_name = "Room"



In [81]:
RESUME = True

In [90]:

import os
from panav.hybrid import reduced_agents_HG
import signal
from time import sleep

def timeout_handler(signum,frame):
    print("Algorithm timeout")
    raise TimeoutError("Timeout")

N_idx = 0
alg_idx = 0
results = []
timeouted = {alg:False for alg in algs}
iteration = 0


def save_checkpoint(env,HG_main,iteration):
    checkpoint = {
        'N_idx':N_idx,
        'alg_idx':alg_idx,
        'results':results,
        'timeouted':timeouted,
        'env':env,
        'HG_main':HG_main,
        'iteration':iteration
    }

    with open(checkpoint_dir+f"_{iteration}_.ckpt",'wb') as fp:
        pkl.dump(checkpoint,fp)
import os
def get_latest_checkpoint(checkpoint_dir):
    ckpts = os.listdir(checkpoint_dir)
    max_iter = -1
    latest_checkpoint = ""
    for ck in ckpts:
        s = ck.split('_')
        # print(int(s[1]))
        if int(s[1])>max_iter:
            max_iter = int(s[1])
            latest_checkpoint = ck
    return latest_checkpoint

import shutil
checkpoint_dir = f"./data/checkpoints/{env_name}/"
if not RESUME:
    if env_name in os.listdir("./data/checkpoints"):
        shutil.rmtree(checkpoint_dir)
    os.mkdir(checkpoint_dir)
    

    env = Room(N_agent=max(n_agents))
    print('Constructing environment')
    HG_main = HybridGraph(env,agent_radius=bloating_r,tunnels=env.tunnels)
    save_checkpoint(env,HG_main,0)
else:
    latest_checkpoint = get_latest_checkpoint(checkpoint_dir)
    with open(checkpoint_dir+latest_checkpoint,'rb') as fp:
        checkpoint = pkl.load(fp)
    N_idx = checkpoint['N_idx']
    alg_idx = checkpoint['alg_idx']
    results = checkpoint['results']
    timeouted = checkpoint['timeouted']
    env = checkpoint['env']
    HG_main = checkpoint['HG_main']
    iteration = checkpoint['iteration']


names = list(algs.keys())

while N_idx<len(n_agents):
    N = n_agents[N_idx]
    HG = reduced_agents_HG(HG_main,N)    
    while True:            
        name = names[alg_idx]
        alg = algs[name]

        if not timeouted[name]:
            
            save_checkpoint(env,HG_main,iteration)
        
            print("Alg:", name, "N agent:",N)

            result = {}
            

            t0 = time()

            signal.signal(signal.SIGALRM, timeout_handler)
            signal.alarm(TIMEOUT)
            try:
                plan = alg(HG)
            except TimeoutError:
                print("Timeout")
            signal.alarm(0)

            t = time()-t0
            
            result['plan'] = plan
            result['time'] = t

            result['N_agent'] = N
            result['alg_name'] = name

            result["TIMEOUT"]=TIMEOUT
            result["HG"]=HG

            results.append(result)

            
            if t>TIMEOUT:
                timeouted[name] = True
                print("Time out for algorithm", name, "N agent = ", N)
            else:
                print("On time for algorithm", name, "N agent = ", N)

        alg_idx += 1
        iteration += 1

        if alg_idx == len(algs):
            alg_idx = 0
            break
    N_idx += 1


Alg: Seq_Tube N agent: 30
Planning for agent 0/30


/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1391: UserWarning: 
    The problem is either infeasible or unbounded, but the solver
    cannot tell which. Disable any solver-specific presolve methods
    and re-solve to determine the precise problem status.

    For GUROBI and CPLEX you can automatically perform this re-solve
    with the keyword argument prob.solve(reoptimize=True, ...).
    
  warnings.warn(INF_OR_UNB_MESSAGE)


Planning for agent 1/30
Planning for agent 2/30
Planning for agent 3/30
Planning for agent 4/30
Planning for agent 5/30
Planning for agent 6/30
Planning for agent 7/30
Planning for agent 8/30
Planning for agent 9/30
Planning for agent 10/30
Planning for agent 11/30
Planning for agent 12/30
Planning for agent 13/30
Planning for agent 14/30
Planning for agent 15/30
Planning for agent 16/30
Planning for agent 17/30
Planning for agent 18/30
Planning for agent 19/30
Planning for agent 20/30
Planning for agent 21/30
Planning for agent 22/30
Planning for agent 23/30
Algorithm timeout
Timeout
Time out for algorithm Seq_Tube N agent =  30
Alg: PIBT N agent: 40
################# Time step 0 ################
Remaining agents 40
################# Time step 1 ################
Remaining agents 40
################# Time step 2 ################
Remaining agents 40
################# Time step 3 ################
Remaining agents 40
################# Time step 4 ################
Remaining agents 40
#####

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/shapely/measurement.py:74: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


################# Time step 116 ################
Remaining agents 43
################# Time step 117 ################
Remaining agents 43
################# Time step 118 ################
Remaining agents 43
################# Time step 119 ################
Remaining agents 43
################# Time step 120 ################
Remaining agents 43
################# Time step 121 ################
Remaining agents 42
################# Time step 122 ################
Remaining agents 42
################# Time step 123 ################
Remaining agents 41
################# Time step 124 ################
Remaining agents 41
################# Time step 125 ################
Remaining agents 40
################# Time step 126 ################
Remaining agents 40
################# Time step 127 ################
Remaining agents 40
################# Time step 128 ################
Remaining agents 40
################# Time step 129 ################
Remaining agents 40
################# Time step 130 ##

/Users/tianpengzhang/opt/miniconda3/envs/PA-NAV/lib/python3.11/site-packages/cvxpy/problems/problem.py:1385: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


################# Time step 143 ################
Remaining agents 50
################# Time step 144 ################
Remaining agents 50
################# Time step 145 ################
Remaining agents 50
################# Time step 146 ################
Remaining agents 50
################# Time step 147 ################
Remaining agents 50
################# Time step 148 ################
Remaining agents 49
################# Time step 149 ################
Remaining agents 49
################# Time step 150 ################
Remaining agents 48
################# Time step 151 ################
Remaining agents 48
################# Time step 152 ################
Remaining agents 47
################# Time step 153 ################
Remaining agents 47
################# Time step 154 ################
Remaining agents 47
################# Time step 155 ################
Remaining agents 47
################# Time step 156 ################
Remaining agents 47
################# Time step 157 ##

Time elasped 5.412101745605469e-05
Time elasped 0.5052628517150879
Time elasped 1.008829116821289
Time elasped 1.5133531093597412
Time elasped 2.0185811519622803
Time elasped 2.5231239795684814
Time elasped 3.0237820148468018
Time elasped 3.52891206741333
Time elasped 4.030996084213257
Time elasped 4.532344818115234
Algorithm timeout
Timeout


0